## بناء نموذج الانحدار اللوجستي - الدرس الرابع

![مخطط معلوماتي عن الانحدار اللوجستي مقابل الانحدار الخطي](../../../../../../translated_images/linear-vs-logistic.ba180bf95e7ee66721ba10ebf2dac2666acbd64a88b003c83928712433a13c7d.ar.png)

#### **[اختبار ما قبل المحاضرة](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### المقدمة

في هذا الدرس الأخير حول الانحدار، وهو أحد تقنيات التعلم الآلي *الكلاسيكية* الأساسية، سنلقي نظرة على الانحدار اللوجستي. يمكنك استخدام هذه التقنية لاكتشاف الأنماط للتنبؤ بالفئات الثنائية. هل هذه الحلوى تحتوي على شوكولاتة أم لا؟ هل هذا المرض معدٍ أم لا؟ هل سيختار هذا العميل هذا المنتج أم لا؟

في هذا الدرس، ستتعلم:

- تقنيات الانحدار اللوجستي

✅ عمّق فهمك للعمل مع هذا النوع من الانحدار في هذا [الوحدة التعليمية](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## المتطلبات الأساسية

بعد العمل مع بيانات القرع، أصبحنا على دراية كافية بها لندرك أن هناك فئة ثنائية يمكننا العمل معها: `Color`.

لنقم ببناء نموذج انحدار لوجستي للتنبؤ بذلك، بناءً على بعض المتغيرات، *ما اللون الذي من المرجح أن يكون عليه القرع المعطى* (برتقالي 🎃 أو أبيض 👻).

> لماذا نتحدث عن التصنيف الثنائي في درس متعلق بالانحدار؟ فقط لسهولة التعبير اللغوي، حيث أن الانحدار اللوجستي هو [في الواقع طريقة تصنيف](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)، وإن كانت تعتمد على الخطية. تعرف على طرق أخرى لتصنيف البيانات في مجموعة الدروس التالية.

بالنسبة لهذا الدرس، سنحتاج إلى الحزم التالية:

- `tidyverse`: [tidyverse](https://www.tidyverse.org/) هو [مجموعة من حزم R](https://www.tidyverse.org/packages) مصممة لجعل علم البيانات أسرع وأسهل وأكثر متعة!

- `tidymodels`: إطار عمل [tidymodels](https://www.tidymodels.org/) هو [مجموعة من الحزم](https://www.tidymodels.org/packages/) للنمذجة والتعلم الآلي.

- `janitor`: حزمة [janitor](https://github.com/sfirke/janitor) توفر أدوات بسيطة لفحص وتنظيف البيانات غير النظيفة.

- `ggbeeswarm`: حزمة [ggbeeswarm](https://github.com/eclarke/ggbeeswarm) توفر طرقًا لإنشاء مخططات بأسلوب "beeswarm" باستخدام ggplot2.

يمكنك تثبيتها باستخدام:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

بدلاً من ذلك، يقوم البرنامج النصي أدناه بالتحقق مما إذا كانت الحزم المطلوبة لإكمال هذه الوحدة موجودة ويقوم بتثبيتها إذا كانت مفقودة.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **تعريف السؤال**

لأغراضنا، سنعبر عن هذا كتصنيف ثنائي: "أبيض" أو "ليس أبيض". هناك أيضًا فئة "مخطط" في بياناتنا، ولكن عدد الحالات فيها قليل جدًا، لذلك لن نستخدمها. على أي حال، تختفي هذه الفئة بمجرد إزالة القيم الفارغة من البيانات.

> 🎃 معلومة ممتعة، أحيانًا نطلق على القرع الأبيض اسم "قرع الأشباح". ليس من السهل نحته، لذلك فهو ليس شائعًا مثل القرع البرتقالي، ولكنه يبدو رائعًا! لذا يمكننا أيضًا إعادة صياغة سؤالنا ليكون: "شبح" أو "ليس شبح". 👻

## **حول الانحدار اللوجستي**

يختلف الانحدار اللوجستي عن الانحدار الخطي، الذي تعلمت عنه سابقًا، في عدة جوانب مهمة.

#### **تصنيف ثنائي**

الانحدار اللوجستي لا يقدم نفس الميزات التي يقدمها الانحدار الخطي. الأول يقدم توقعًا حول `فئة ثنائية` ("برتقالي أو ليس برتقالي")، بينما الثاني قادر على توقع `قيم مستمرة`، على سبيل المثال، بناءً على مصدر القرع ووقت الحصاد، *كم ستزداد قيمته*.

![رسم توضيحي بواسطة Dasani Madipalli](../../../../../../translated_images/pumpkin-classifier.562771f104ad5436b87d1c67bca02a42a17841133556559325c0a0e348e5b774.ar.png)

### تصنيفات أخرى

هناك أنواع أخرى من الانحدار اللوجستي، بما في ذلك متعدد الفئات والرتبي:

- **متعدد الفئات**، والذي يتضمن وجود أكثر من فئة واحدة - "برتقالي، أبيض، ومخطط".

- **رتبي**، والذي يتضمن فئات مرتبة، وهو مفيد إذا أردنا ترتيب النتائج بشكل منطقي، مثل القرع الذي يتم ترتيبه حسب عدد محدود من الأحجام (صغير جدًا، صغير، متوسط، كبير، كبير جدًا، كبير جدًا جدًا).

![الانحدار متعدد الفئات مقابل الرتبي](../../../../../../translated_images/multinomial-vs-ordinal.36701b4850e37d86c9dd49f7bef93a2f94dbdb8fe03443eb68f0542f97f28f29.ar.png)

#### **المتغيرات لا يجب أن تكون مترابطة**

هل تتذكر كيف كان الانحدار الخطي يعمل بشكل أفضل مع المتغيرات الأكثر ترابطًا؟ الانحدار اللوجستي عكس ذلك - المتغيرات لا تحتاج إلى أن تكون متوافقة. وهذا يناسب هذه البيانات التي تحتوي على ترابطات ضعيفة إلى حد ما.

#### **تحتاج إلى الكثير من البيانات النظيفة**

سيعطي الانحدار اللوجستي نتائج أكثر دقة إذا استخدمت المزيد من البيانات؛ مجموعتنا الصغيرة من البيانات ليست مثالية لهذه المهمة، لذا ضع ذلك في اعتبارك.

✅ فكر في أنواع البيانات التي يمكن أن تكون مناسبة للانحدار اللوجستي

## تمرين - ترتيب البيانات

أولاً، قم بتنظيف البيانات قليلاً، بإزالة القيم الفارغة واختيار بعض الأعمدة فقط:

1. أضف الكود التالي:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


يمكنك دائمًا إلقاء نظرة سريعة على إطار البيانات الجديد الخاص بك باستخدام وظيفة [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) كما هو موضح أدناه:


In [ ]:
pumpkins_select %>% 
  glimpse()


دعونا نتأكد من أننا سنقوم فعلاً بحل مشكلة تصنيف ثنائي:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### التصور - مخطط الفئات
بحلول هذه المرحلة، قمت بتحميل بيانات القرع مرة أخرى وتنظيفها للحفاظ على مجموعة بيانات تحتوي على بعض المتغيرات، بما في ذلك اللون. دعنا نتصور إطار البيانات في الدفتر باستخدام مكتبة ggplot.

تقدم مكتبة ggplot طرقًا رائعة لتصور بياناتك. على سبيل المثال، يمكنك مقارنة توزيعات البيانات لكل نوع ولون في مخطط الفئات.

1. قم بإنشاء مثل هذا المخطط باستخدام وظيفة geombar، مع استخدام بيانات القرع الخاصة بنا، وتحديد تعيين اللون لكل فئة من فئات القرع (البرتقالي أو الأبيض):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

من خلال ملاحظة البيانات، يمكنك أن ترى كيف ترتبط بيانات اللون بالتنوع.

✅ بالنظر إلى هذا المخطط التصنيفي، ما هي بعض الاستكشافات المثيرة للاهتمام التي يمكنك تصورها؟


### معالجة البيانات: ترميز الميزات

تحتوي مجموعة بيانات القرع الخاصة بنا على قيم نصية في جميع أعمدتها. التعامل مع البيانات الفئوية أمر بديهي للبشر ولكنه ليس كذلك للآلات. تعمل خوارزميات التعلم الآلي بشكل أفضل مع الأرقام. لهذا السبب، يُعد الترميز خطوة مهمة جدًا في مرحلة معالجة البيانات، حيث يتيح لنا تحويل البيانات الفئوية إلى بيانات رقمية دون فقدان أي معلومات. الترميز الجيد يؤدي إلى بناء نموذج جيد.

هناك نوعان رئيسيان من أدوات الترميز لترميز الميزات:

1. **الترميز الترتيبي (Ordinal encoder):** يناسب المتغيرات الترتيبية، وهي المتغيرات الفئوية التي تتبع بياناتها ترتيبًا منطقيًا، مثل عمود `item_size` في مجموعة بياناتنا. يقوم بإنشاء خريطة بحيث يتم تمثيل كل فئة برقم يمثل ترتيب الفئة في العمود.

2. **الترميز الفئوي (Categorical encoder):** يناسب المتغيرات الاسمية، وهي المتغيرات الفئوية التي لا تتبع بياناتها ترتيبًا منطقيًا، مثل جميع الميزات الأخرى غير `item_size` في مجموعة بياناتنا. يتمثل في الترميز الثنائي (one-hot encoding)، مما يعني أن كل فئة يتم تمثيلها بعمود ثنائي: تكون القيمة المشفرة مساوية لـ 1 إذا كان القرع ينتمي إلى تلك الفئة و0 خلاف ذلك.

يوفر Tidymodels حزمة أخرى رائعة: [recipes](https://recipes.tidymodels.org/) - وهي حزمة لمعالجة البيانات. سنقوم بتعريف `recipe` يحدد أن جميع أعمدة المتنبئات يجب أن يتم ترميزها إلى مجموعة من الأعداد الصحيحة، ثم نقوم بـ `prep` لتقدير الكميات والإحصائيات المطلوبة لأي عمليات، وأخيرًا `bake` لتطبيق العمليات على بيانات جديدة.

> عادةً، يتم استخدام recipes كمعالج مسبق للنمذجة، حيث يحدد الخطوات التي يجب تطبيقها على مجموعة بيانات لجعلها جاهزة للنمذجة. في هذه الحالة، **ننصح بشدة** باستخدام `workflow()` بدلاً من تقدير الوصفة يدويًا باستخدام prep وbake. سنرى كل هذا بعد قليل.
>
> ولكن في الوقت الحالي، نحن نستخدم recipes + prep + bake لتحديد الخطوات التي يجب تطبيقها على مجموعة بيانات لجعلها جاهزة لتحليل البيانات، ثم استخراج البيانات المعالجة مسبقًا مع الخطوات المطبقة.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ ما هي فوائد استخدام الترميز الترتيبي لعمود حجم العنصر؟

### تحليل العلاقات بين المتغيرات

الآن بعد أن قمنا بمعالجة البيانات مسبقًا، يمكننا تحليل العلاقات بين الميزات والتسمية لفهم مدى قدرة النموذج على التنبؤ بالتسمية بناءً على الميزات. أفضل طريقة لإجراء هذا النوع من التحليل هي رسم البيانات.  
سنستخدم مرة أخرى وظيفة ggplot geom_boxplot_، لتصور العلاقات بين حجم العنصر، النوع، واللون في مخطط تصنيفي. لتحسين رسم البيانات، سنستخدم عمود حجم العنصر المشفر وعمود النوع غير المشفر.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### استخدام مخطط السرب

بما أن اللون هو فئة ثنائية (أبيض أو غير ذلك)، فإنه يحتاج إلى "[نهج متخصص](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)" للتصور.

جرّب `مخطط السرب` لعرض توزيع اللون بالنسبة لحجم العنصر.

سنستخدم [حزمة ggbeeswarm](https://github.com/eclarke/ggbeeswarm) التي توفر طرقًا لإنشاء مخططات بأسلوب السرب باستخدام ggplot2. تُعد مخططات السرب وسيلة لعرض النقاط التي قد تتداخل عادةً بحيث تقع بجانب بعضها البعض بدلاً من ذلك.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


الآن بعد أن أصبح لدينا فكرة عن العلاقة بين الفئات الثنائية للألوان والمجموعة الأكبر من الأحجام، دعونا نستكشف الانحدار اللوجستي لتحديد اللون المحتمل لليقطينة.

## بناء النموذج الخاص بك

حدد المتغيرات التي تريد استخدامها في نموذج التصنيف الخاص بك وقم بتقسيم البيانات إلى مجموعات تدريب واختبار. [rsample](https://rsample.tidymodels.org/)، وهي حزمة في Tidymodels، توفر بنية تحتية لتقسيم البيانات وإعادة أخذ العينات بكفاءة:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 نحن الآن جاهزون لتدريب نموذج من خلال ملاءمة ميزات التدريب مع تسمية التدريب (اللون).

سنبدأ بإنشاء وصفة تحدد خطوات المعالجة المسبقة التي يجب تنفيذها على بياناتنا لتجهيزها للنمذجة، مثل: ترميز المتغيرات الفئوية إلى مجموعة من الأعداد الصحيحة. تمامًا مثل `baked_pumpkins`، نقوم بإنشاء `pumpkins_recipe` ولكن لا نقوم بـ `prep` و `bake` لأن ذلك سيتم تضمينه في سير العمل، والذي ستراه في بضع خطوات من الآن.

هناك العديد من الطرق لتحديد نموذج الانحدار اللوجستي في Tidymodels. انظر `?logistic_reg()` في الوقت الحالي، سنحدد نموذج الانحدار اللوجستي عبر محرك `stats::glm()` الافتراضي.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


الآن بعد أن أصبح لدينا وصفة ومواصفات نموذج، نحتاج إلى إيجاد طريقة لدمجهما معًا في كائن يقوم أولاً بمعالجة البيانات مسبقًا (التحضير + الخبز خلف الكواليس)، ثم تدريب النموذج على البيانات المعالجة مسبقًا، وأيضًا يتيح إمكانية القيام بأنشطة معالجة لاحقة.

في Tidymodels، يُطلق على هذا الكائن المريح اسم [`workflow`](https://workflows.tidymodels.org/) وهو يحتفظ بشكل ملائم بمكونات النمذجة الخاصة بك.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


بعد تحديد سير العمل، يمكن تدريب النموذج باستخدام وظيفة [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html). سيقوم سير العمل بتقدير الوصفة ومعالجة البيانات مسبقًا قبل التدريب، لذلك لن نحتاج إلى القيام بذلك يدويًا باستخدام prep و bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


يعرض النموذج القيم التي تم تعلمها أثناء التدريب.

الآن بعد أن قمنا بتدريب النموذج باستخدام بيانات التدريب، يمكننا إجراء التنبؤات على بيانات الاختبار باستخدام [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). لنبدأ باستخدام النموذج لتوقع التصنيفات لمجموعة الاختبار واحتمالات كل تصنيف. عندما تكون الاحتمالية أكبر من 0.5، يكون التصنيف المتوقع هو `WHITE`، وإلا يكون `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


جميل جدًا! هذا يوفر بعض الأفكار الإضافية حول كيفية عمل الانحدار اللوجستي.

### فهم أفضل من خلال مصفوفة الالتباس

مقارنة كل توقع بالقيمة "الحقيقية" المقابلة له ليست طريقة فعالة جدًا لتحديد مدى دقة توقعات النموذج. لحسن الحظ، توفر مكتبة Tidymodels بعض الأدوات الإضافية: [`yardstick`](https://yardstick.tidymodels.org/) - وهي حزمة تُستخدم لقياس فعالية النماذج باستخدام مقاييس الأداء.

أحد مقاييس الأداء المرتبطة بمشاكل التصنيف هو [`مصفوفة الالتباس`](https://wikipedia.org/wiki/Confusion_matrix). تصف مصفوفة الالتباس مدى كفاءة أداء نموذج التصنيف. تقوم مصفوفة الالتباس بتبويب عدد الأمثلة في كل فئة التي تم تصنيفها بشكل صحيح بواسطة النموذج. في حالتنا، ستوضح لك عدد القرعات البرتقالية التي تم تصنيفها كبرتقالية وعدد القرعات البيضاء التي تم تصنيفها كبيضاء؛ كما تُظهر مصفوفة الالتباس أيضًا عدد الحالات التي تم تصنيفها في الفئات **الخاطئة**.

وظيفة [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) من حزمة yardstick تحسب هذا الجدول المتقاطع للفئات الملاحظة والمتوقعة.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


دعونا نفسر مصفوفة الالتباس. يُطلب من نموذجنا تصنيف القرع بين فئتين ثنائيتين، الفئة `أبيض` والفئة `غير أبيض`.

- إذا توقع نموذجك أن القرع أبيض وكان ينتمي بالفعل إلى الفئة "أبيض"، فإننا نسميه `إيجابي حقيقي`، ويظهر في الرقم العلوي الأيسر.

- إذا توقع نموذجك أن القرع غير أبيض وكان ينتمي بالفعل إلى الفئة "أبيض"، فإننا نسميه `سلبي خاطئ`، ويظهر في الرقم السفلي الأيسر.

- إذا توقع نموذجك أن القرع أبيض وكان ينتمي بالفعل إلى الفئة "غير أبيض"، فإننا نسميه `إيجابي خاطئ`، ويظهر في الرقم العلوي الأيمن.

- إذا توقع نموذجك أن القرع غير أبيض وكان ينتمي بالفعل إلى الفئة "غير أبيض"، فإننا نسميه `سلبي حقيقي`، ويظهر في الرقم السفلي الأيمن.

| الحقيقة |
|:-----:|

|               |        |       |
|---------------|--------|-------|
| **التوقع**    | أبيض   | برتقالي |
| أبيض          | TP     | FP    |
| برتقالي       | FN     | TN    |

كما قد تكون خمنت، من الأفضل أن يكون لدينا عدد أكبر من الإيجابيات الحقيقية والسلبيات الحقيقية وعدد أقل من الإيجابيات الخاطئة والسلبيات الخاطئة، مما يعني أن النموذج يعمل بشكل أفضل.

مصفوفة الالتباس مفيدة لأنها تؤدي إلى مقاييس أخرى يمكن أن تساعدنا في تقييم أداء نموذج التصنيف بشكل أفضل. دعونا نستعرض بعضها:

🎓 الدقة: `TP/(TP + FP)` تُعرف بأنها نسبة الإيجابيات المتوقعة التي هي بالفعل إيجابية. تُسمى أيضًا [القيمة التنبؤية الإيجابية](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 الاستدعاء: `TP/(TP + FN)` يُعرف بأنه نسبة النتائج الإيجابية من إجمالي العينات التي كانت بالفعل إيجابية. يُعرف أيضًا بـ `الحساسية`.

🎓 الخصوصية: `TN/(TN + FP)` تُعرف بأنها نسبة النتائج السلبية من إجمالي العينات التي كانت بالفعل سلبية.

🎓 الدقة العامة: `TP + TN/(TP + TN + FP + FN)` النسبة المئوية للتصنيفات التي تم توقعها بشكل صحيح لعينة.

🎓 مقياس F: متوسط مرجح للدقة والاستدعاء، حيث الأفضل هو 1 والأسوأ هو 0.

دعونا نحسب هذه المقاييس!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## تصور منحنى ROC لهذا النموذج

دعونا نقوم بتصور إضافي لرؤية ما يُعرف بـ [`منحنى ROC`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


منحنيات ROC تُستخدم غالبًا للحصول على نظرة على مخرجات المصنف من حيث الإيجابيات الحقيقية مقابل الإيجابيات الكاذبة. عادةً ما تعرض منحنيات ROC معدل الإيجابيات الحقيقية/الحساسية على المحور الرأسي، ومعدل الإيجابيات الكاذبة/1-النوعية على المحور الأفقي. وبالتالي، فإن انحدار المنحنى والمسافة بين خط المنتصف والمنحنى لهما أهمية: الهدف هو الحصول على منحنى يتجه بسرعة إلى الأعلى ويتجاوز الخط. في حالتنا، هناك إيجابيات كاذبة في البداية، ثم يتجه الخط إلى الأعلى ويتجاوز بشكل صحيح.

أخيرًا، دعونا نستخدم `yardstick::roc_auc()` لحساب المساحة الفعلية تحت المنحنى. إحدى طرق تفسير AUC هي أنها تمثل احتمال أن يصنف النموذج مثالًا إيجابيًا عشوائيًا بدرجة أعلى من مثال سلبي عشوائي.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


النتيجة حوالي `0.975`. نظرًا لأن قيمة AUC تتراوح بين 0 و1، فإنك تريد الحصول على درجة عالية، لأن النموذج الذي يكون دقيقًا بنسبة 100% في توقعاته سيحصل على AUC بقيمة 1؛ في هذه الحالة، النموذج *جيد جدًا*.

في الدروس المستقبلية حول التصنيفات، ستتعلم كيفية تحسين درجات النموذج (مثل التعامل مع البيانات غير المتوازنة في هذه الحالة).

## 🚀تحدي

هناك الكثير لاستكشافه فيما يتعلق بالانحدار اللوجستي! ولكن أفضل طريقة للتعلم هي التجربة. ابحث عن مجموعة بيانات تناسب هذا النوع من التحليل وقم ببناء نموذج باستخدامها. ماذا تتعلم؟ نصيحة: جرّب [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) للحصول على مجموعات بيانات مثيرة للاهتمام.

## المراجعة والدراسة الذاتية

اقرأ الصفحات الأولى من [هذه الورقة البحثية من جامعة ستانفورد](https://web.stanford.edu/~jurafsky/slp3/5.pdf) حول بعض الاستخدامات العملية للانحدار اللوجستي. فكّر في المهام التي تناسب نوعًا معينًا من مهام الانحدار التي درسناها حتى الآن. ما الذي سيكون الأنسب؟



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
